In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

In [2]:
bdname='sinesearch_db'
bduser = 'postgres'
bdpassword = 'nef441'
bdhost = '51.79.69.179'
bdport = 5432

db = PostgresqlExtDatabase(bdname, user=bduser, password=bdpassword,
                           host=bdhost, port=bdport)

In [5]:
db.rollback()

AttributeError: 'NoneType' object has no attribute 'rollback'

In [3]:
class Films(Model):
    film_id=IntegerField(unique=True,index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube=TextField(default="")
    stars = FloatField(default=0)
    info=JSONField(default={})
    selections=ArrayField(IntegerField,default=[])
    likes=IntegerField(default=0)
    dislikes=IntegerField(default=0)

    class Meta:
        database = db
        db_table='Films'

class Selections(Model):
    name = TextField()
    user_id = IntegerField(default=0,index=True)
    stars = FloatField(default=0)
    films = ArrayField(IntegerField,default=[])

    class Meta:
        database = db
        db_table='Selections'

In [4]:
with open("collections","r") as f:
    collections=json.load(f)

In [5]:
col=[]
for i , c in tqdm(enumerate(collections)):
    col.append({
         "name":c,
         "films":[int(k) for k in collections[c]]
     })
    if i % 100 ==0:
        Selections.insert_many(col).execute()
        col=[]
Selections.insert_many(col).execute()

In [8]:
re.match(r'vk\.com/.*', 'k.com/p.volnov')

In [25]:
for i , col in enumerate(collections.values()):
    print(col)
    break

['434835', '23093', '433037', '419841', '421800', '413729', '23976', '226247', '419328', '8941', '415110', '23491', '2866', '4465', '21663', '422195', '1201', '242343', '10241', '415684', '4944', '199635', '416039', '380598']


In [26]:
fms=[]
with open("films.jsonlines","r") as f:
    for k, line in enumerate(f.readlines()[:120]):
        film = json.loads(line)
        info=film.copy()
        comp_film={
            "name":info["name"],
            "discr":info["discr"],
            "img":info["img"],
            "youtube":info["youtube"],
            "film_id":info["film_id"],   
            "stars":0
        }
        if "Рейтинг" in info:
            comp_film["stars"] = float(info["Рейтинг"].replace(" ","").replace(",",".").replace("%",""))/10
        del info["name"]
        del info["discr"]
        del info["img"]
        del info["youtube"]
        del info["film_id"]
        info["Дата выхода"]=info["data"]
        del info["data"]

        
        comp_film["selections"]=[]
        
        for i , col in enumerate(collections.values()):
            if str(film["film_id"]) in col:
                comp_film["selections"].append(i+1)

        comp_film["info"]=info
        
        fms.append(comp_film)
        if k % 50 == 0:
            Films.insert_many(fms).execute()   
            print(len(fms))
            fms=[]
Films.insert_many(fms).execute()

1
50
50


In [2]:
import re
re.match(r'vk\.com/.*', "vk.com/neafiol")==None

False